# Panel 1

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # For PCA
            library(DESeq2), 
            
            # Data
            library(dplyr), 

            # STRING
            library(STRINGdb), 
            library(igraph), 
            library(tidygraph), 
            library(ggraph), 
            
            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(ComplexHeatmap), 
            library(grid) # For gpar
            
        )
    )
)

In [ ]:
options(warn=-1)

In [ ]:
# Set working directory to project root
setwd("C:/Users/Florian Deckert/Projects/FD20200109SPLENO/") 

In [ ]:
# Source
source("bin/so_pl.R")
source("bin/dea_pl.R")
source("bin/gsea_pp.R")
source("bin/gsea_pl.R")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Import and parameter 

In [ ]:
singler <- read.csv("result/singler/bulkRNAseq/singler_res.csv", row.names=1)

In [ ]:
so <- readRDS("data/scRNAseq/object/pp_0.rds")
so <- AddMetaData(so, singler)

In [ ]:
so <- NormalizeData(so)

# SingleR UMAP

In [ ]:
fp_1 <- list(

    fplot(so, features="cmo_score", pt_size=0.25, min_cutoff=0, order=TRUE, restrict=c("sample_group", "Bl6_NaCl_D6"), size_select=4), 
    fplot(so, features="cmo_score", pt_size=0.25, min_cutoff=0, order=TRUE, restrict=c("sample_group", "Bl6_CpG_D6"), size_select=4), 
    fplot(so, features="rpm_score", pt_size=0.25, min_cutoff=0, order=TRUE, restrict=c("sample_group", "Bl6_NaCl_D6"), size_select=4), 
    fplot(so, features="rpm_score", pt_size=0.25, min_cutoff=0, order=TRUE, restrict=c("sample_group", "Bl6_CpG_D6"), size_select=4), 
    fplot(so, features="Spic", pt_size=0.25, min_cutoff=0, order=TRUE, slot="data", assay="RNA", restrict=c("sample_group", "Bl6_NaCl_D6"), size_select=4), 
    fplot(so, features="Spic", pt_size=0.25, min_cutoff=0, order=TRUE, slot="data", assay="RNA", restrict=c("sample_group", "Bl6_CpG_D6"), size_select=4), 
    fplot(so, features="Gdf15", pt_size=0.25, min_cutoff=0, order=TRUE, slot="data", assay="RNA", restrict=c("sample_group", "Bl6_NaCl_D6"), size_select=4), 
    fplot(so, features="Gdf15", pt_size=0.25, min_cutoff=0, order=TRUE, slot="data", assay="RNA", restrict=c("sample_group", "Bl6_CpG_D6"), size_select=4)

    
)

In [ ]:
fp_1 <- lapply(fp_1, function(p) egg::set_panel_size(p, width=unit(2.5, "cm"), height=unit(2.5, "cm")))
fp_1 <- do.call(gridExtra::arrangeGrob, c(fp_1, ncol=2, nrow=length(fp_1)/2))

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/fp_rpm_score.pdf", width=2*2.0, height=1.5*length(fp_1)/2)

grid::grid.draw(fp_1)

dev.off()

# DEA result count heatmap

In [ ]:
adj_pval_thr <- 0.05
log2FC_thr <- 0.50

## Fetch results

In [ ]:
res_dea_1 <- readRDS("result/dea/bulkRNAseq/res_dea_cmo.rds")
res_dea_2 <- readRDS("result/dea/bulkRNAseq/res_dea_rpm.rds")
res_dea_3 <- readRDS("result/dea/bulkRNAseq/res_dea_ery.rds")

In [ ]:
dpi_lrt <- dplyr::left_join(

    do.call(rbind, 
    
        list(
            
            res_dea_1[["res"]][["dpi_wt_lrt"]] %>% dplyr::mutate(celltype="cMo") %>% tibble::rownames_to_column("gene"), 
            
            res_dea_2[["res"]][["dpi_wt_lrt"]] %>% dplyr::mutate(celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            
            res_dea_3[["res"]][["dpi_wt_lrt"]] %>% dplyr::mutate(celltype="Ery") %>% tibble::rownames_to_column("gene")
        
            )
    
        ) %>% dplyr::select(gene, log2FoldChange, ihw, celltype) %>% dplyr::rename(dpi_wt_lrt_fc=log2FoldChange, dpi_wt_lrt_ihw=ihw), 

    do.call(rbind, 
    
        list(
            
            res_dea_1[["res"]][["dpi_ko_lrt"]] %>% dplyr::mutate(celltype="cMo") %>% tibble::rownames_to_column("gene"), 
            
            res_dea_2[["res"]][["dpi_ko_lrt"]] %>% dplyr::mutate(celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            
            res_dea_3[["res"]][["dpi_ko_lrt"]] %>% dplyr::mutate(celltype="Ery") %>% tibble::rownames_to_column("gene")
        
            )
    
        ) %>% dplyr::select(gene, log2FoldChange, ihw, celltype) %>% dplyr::rename(dpi_ko_lrt_fc=log2FoldChange, dpi_ko_lrt_ihw=ihw), 


    by=join_by(gene, celltype)

) 

In [ ]:
dpi_wald <- dplyr::left_join(

    do.call(rbind, 
    
        list(
            
            res_dea_1[["res"]][["dpi_wt_wald"]][[1]] %>% dplyr::mutate(dpi="D1", celltype="cMo") %>% tibble::rownames_to_column("gene"), 
            res_dea_1[["res"]][["dpi_wt_wald"]][[2]] %>% dplyr::mutate(dpi="D3", celltype="cMo") %>% tibble::rownames_to_column("gene"), 
            res_dea_1[["res"]][["dpi_wt_wald"]][[3]] %>% dplyr::mutate(dpi="D6", celltype="cMo") %>% tibble::rownames_to_column("gene"), 
            
            res_dea_2[["res"]][["dpi_wt_wald"]][[1]] %>% dplyr::mutate(dpi="D1", celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            res_dea_2[["res"]][["dpi_wt_wald"]][[2]] %>% dplyr::mutate(dpi="D3", celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            res_dea_2[["res"]][["dpi_wt_wald"]][[3]] %>% dplyr::mutate(dpi="D6", celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            
            res_dea_3[["res"]][["dpi_wt_wald"]][[1]] %>% dplyr::mutate(dpi="D1", celltype="Ery") %>% tibble::rownames_to_column("gene"), 
            res_dea_3[["res"]][["dpi_wt_wald"]][[2]] %>% dplyr::mutate(dpi="D3", celltype="Ery") %>% tibble::rownames_to_column("gene"), 
            res_dea_3[["res"]][["dpi_wt_wald"]][[3]] %>% dplyr::mutate(dpi="D6", celltype="Ery") %>% tibble::rownames_to_column("gene")
        
            )
    
        ) %>% dplyr::select(gene, log2FoldChange, ihw, dpi, celltype) %>% dplyr::rename(dpi_wt_wald_fc=log2FoldChange, dpi_wt_wald_ihw=ihw), 

    do.call(rbind, 
    
        list(
            
            res_dea_1[["res"]][["dpi_ko_wald"]][[1]] %>% dplyr::mutate(dpi="D1", celltype="cMo") %>% tibble::rownames_to_column("gene"), 
            res_dea_1[["res"]][["dpi_ko_wald"]][[2]] %>% dplyr::mutate(dpi="D3", celltype="cMo") %>% tibble::rownames_to_column("gene"), 
            res_dea_1[["res"]][["dpi_ko_wald"]][[3]] %>% dplyr::mutate(dpi="D6", celltype="cMo") %>% tibble::rownames_to_column("gene"), 

            res_dea_2[["res"]][["dpi_ko_wald"]][[1]] %>% dplyr::mutate(dpi="D1", celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            res_dea_2[["res"]][["dpi_ko_wald"]][[2]] %>% dplyr::mutate(dpi="D3", celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            res_dea_2[["res"]][["dpi_ko_wald"]][[3]] %>% dplyr::mutate(dpi="D6", celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            
            res_dea_3[["res"]][["dpi_ko_wald"]][[1]] %>% dplyr::mutate(dpi="D1", celltype="Ery") %>% tibble::rownames_to_column("gene"), 
            res_dea_3[["res"]][["dpi_ko_wald"]][[2]] %>% dplyr::mutate(dpi="D3", celltype="Ery") %>% tibble::rownames_to_column("gene"), 
            res_dea_3[["res"]][["dpi_ko_wald"]][[3]] %>% dplyr::mutate(dpi="D6", celltype="Ery") %>% tibble::rownames_to_column("gene")
        
            )
    
        ) %>% dplyr::select(gene, log2FoldChange, ihw, dpi, celltype) %>% dplyr::rename(dpi_ko_wald_fc=log2FoldChange, dpi_ko_wald_ihw=ihw), 


    by=join_by(gene, dpi, celltype)

) 

In [ ]:
dpi_genotype_wald <- do.call(rbind, 
    
        list(
            
            res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]][[2]] %>% dplyr::mutate(dpi="D1", celltype="cMo") %>% tibble::rownames_to_column("gene"), 
            res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]][[3]] %>% dplyr::mutate(dpi="D3", celltype="cMo") %>% tibble::rownames_to_column("gene"), 
            res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]][[4]] %>% dplyr::mutate(dpi="D6", celltype="cMo") %>% tibble::rownames_to_column("gene"), 
            
            res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]][[2]] %>% dplyr::mutate(dpi="D1", celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]][[3]] %>% dplyr::mutate(dpi="D3", celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]][[4]] %>% dplyr::mutate(dpi="D6", celltype="RPM") %>% tibble::rownames_to_column("gene"), 
            
            res_dea_3[["res"]][["dpi_wt_vs_ko_wald"]][[2]] %>% dplyr::mutate(dpi="D1", celltype="Ery") %>% tibble::rownames_to_column("gene"), 
            res_dea_3[["res"]][["dpi_wt_vs_ko_wald"]][[3]] %>% dplyr::mutate(dpi="D3", celltype="Ery") %>% tibble::rownames_to_column("gene"), 
            res_dea_3[["res"]][["dpi_wt_vs_ko_wald"]][[4]] %>% dplyr::mutate(dpi="D6", celltype="Ery") %>% tibble::rownames_to_column("gene")
        
            )
    
        ) %>% dplyr::select(gene, log2FoldChange, ihw, dpi, celltype) %>% dplyr::rename(genotype_wt_vs_ko_wald_fc=log2FoldChange, genotype_wt_vs_ko_wald_ihw=ihw)

In [ ]:
data <- dplyr::left_join(dpi_genotype_wald, dpi_wald, by=join_by(gene, dpi, celltype)) %>% dplyr::left_join(., dpi_lrt, by=join_by(gene, celltype))

## Response genes (jitter plot)

In [ ]:
mat_1 <- rbind(

    data %>% dplyr::select(dpi, gene, celltype, dpi_wt_wald_fc, dpi_wt_wald_ihw, dpi_wt_lrt_fc, dpi_wt_lrt_ihw) %>% dplyr::rename(dpi_wald_fc=dpi_wt_wald_fc, dpi_wald_ihw=dpi_wt_wald_ihw, dpi_lrt_fc=dpi_wt_lrt_fc, dpi_lrt_ihw=dpi_wt_lrt_ihw) %>% dplyr::mutate(genotype="+/+"), 
    data %>% dplyr::select(dpi, gene, celltype, dpi_ko_wald_fc, dpi_ko_wald_ihw, dpi_ko_lrt_fc, dpi_ko_lrt_ihw) %>% dplyr::rename(dpi_wald_fc=dpi_ko_wald_fc, dpi_wald_ihw=dpi_ko_wald_ihw, dpi_lrt_fc=dpi_ko_lrt_fc, dpi_lrt_ihw=dpi_ko_lrt_ihw) %>% dplyr::mutate(genotype="cre/+")
    
) %>% 
    dplyr::mutate(sig=ifelse(genotype=="+/+" & dpi_wald_ihw<=adj_pval_thr & abs(dpi_wald_fc)>=log2FC_thr & dpi_lrt_ihw<=adj_pval_thr, "+/+", ifelse(genotype=="cre/+" & dpi_wald_ihw<=adj_pval_thr & abs(dpi_wald_fc)>=log2FC_thr & dpi_lrt_ihw<=adj_pval_thr, "cre/+", "n.s."))) %>% 
    dplyr::mutate(celltype=factor(celltype, levels=c("cMo", "RPM", "Ery"))) %>% na.omit()

In [ ]:
# UP-only comparison
up_counts <- mat_1 %>% dplyr::mutate(sign=ifelse(dpi_wald_fc>0, "(UP)", "(DOWN)")) %>% dplyr::filter(sig!="n.s.") %>% dplyr::select(gene, dpi, celltype, sig, sign) %>%
  filter(sig %in% c("+/+", "cre/+"), sign == "(UP)") %>%
  distinct(gene, dpi, celltype, sig) %>%
  mutate(present = 1L) %>%
  pivot_wider(names_from = sig, values_from = present, values_fill = 0L) %>%
  mutate(category = case_when(
    `+/+` == 1L & `cre/+` == 1L ~ "UP_shared",
    `+/+` == 1L & `cre/+` == 0L ~ "UP_+/+_only",
    `+/+` == 0L & `cre/+` == 1L ~ "UP_cre/+_only"
  )) %>%
  count(dpi, celltype, category)

# DOWN-only comparison
down_counts <- mat_1 %>% dplyr::mutate(sign=ifelse(dpi_wald_fc>0, "(UP)", "(DOWN)")) %>% dplyr::filter(sig!="n.s.") %>% dplyr::select(gene, dpi, celltype, sig, sign) %>%
  filter(sig %in% c("+/+", "cre/+"), sign == "(DOWN)") %>%
  distinct(gene, dpi, celltype, sig) %>%
  mutate(present = 1L) %>%
  pivot_wider(names_from = sig, values_from = present, values_fill = 0L) %>%
  mutate(category = case_when(
    `+/+` == 1L & `cre/+` == 1L ~ "DOWN_shared",
    `+/+` == 1L & `cre/+` == 0L ~ "DOWN_+/+_only",
    `+/+` == 0L & `cre/+` == 1L ~ "DOWN_cre/+_only"
  )) %>%
  count(dpi, celltype, category)

counts_1 <- bind_rows(up_counts, down_counts) %>% arrange(dpi, celltype, category)

In [ ]:
mat_1 <- mat_1 %>% dplyr::mutate(sign_log10_padj=-sign(dpi_wald_fc)*log10(dpi_wald_ihw)) %>% dplyr::mutate(sign_log10_padj=ifelse(abs(sign_log10_padj)>60, sign(dpi_wald_fc)*60, sign_log10_padj)) %>% 
    dplyr::mutate(dpi_wald_fc=ifelse(abs(dpi_wald_fc)>15, sign(dpi_wald_fc)*15, dpi_wald_fc))

In [ ]:
jp_1 <- ggplot(mat_1 %>% dplyr::filter(celltype=="cMo"), aes(y=sign_log10_padj, x=genotype, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="cMo" & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="cMo" & sig!="n.s."), size=0.1) + 
    geom_hline(yintercept=0) +
    ylim(-60, +60) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

jp_2 <- ggplot(mat_1 %>% dplyr::filter(celltype=="RPM"), aes(y=sign_log10_padj, x=genotype, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="RPM" & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="RPM" & sig!="n.s."), size=0.1) + 
    geom_hline(yintercept=0) +
    ylim(-60, +60) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

jp_3 <- ggplot(mat_1 %>% dplyr::filter(celltype=="Ery"), aes(y=sign_log10_padj, x=genotype, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="Ery" & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="Ery" & sig!="n.s."), size=0.1) +  
    geom_hline(yintercept=0) +
    ylim(-60, +60) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/jp_dea_count_dpi.pdf", width=3, height=4)

    gridExtra::grid.arrange(
            
        jp_1 %>% egg::set_panel_size(., width=unit(1, "cm"), height=unit(1.5, "cm")), 
        jp_2 %>% egg::set_panel_size(., width=unit(1, "cm"), height=unit(1.5, "cm")), 
        jp_3 %>% egg::set_panel_size(., width=unit(1, "cm"), height=unit(1.5, "cm")), 
        nrow=3
        
    )

dev.off()

## Genotype perturbation 

In [ ]:
mat_1 <- data %>% dplyr::select(gene, genotype_wt_vs_ko_wald_fc, genotype_wt_vs_ko_wald_ihw, dpi, celltype) %>% 
    dplyr::mutate(sig=ifelse(genotype_wt_vs_ko_wald_ihw<=adj_pval_thr & genotype_wt_vs_ko_wald_fc<=-log2FC_thr, "+/+", ifelse(genotype_wt_vs_ko_wald_ihw<=adj_pval_thr & genotype_wt_vs_ko_wald_fc>=+log2FC_thr, "cre/+", "n.s."))) 

In [ ]:
mat_1 <- mat_1 %>% dplyr::mutate(sign_log10_padj=-sign(genotype_wt_vs_ko_wald_fc)*log10(genotype_wt_vs_ko_wald_ihw)) %>% dplyr::mutate(sign_log10_padj=ifelse(abs(sign_log10_padj)>10, sign(genotype_wt_vs_ko_wald_fc)*10, sign_log10_padj)) %>% 
    dplyr::mutate(genotype_wt_vs_ko_wald_fc=ifelse(abs(genotype_wt_vs_ko_wald_fc)>10, sign(genotype_wt_vs_ko_wald_fc)*10, genotype_wt_vs_ko_wald_fc))

In [ ]:
jp_1 <- ggplot(mat_1 %>% dplyr::filter(celltype=="cMo"), aes(y=sign_log10_padj, x=dpi, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="cMo" & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="cMo" & sig!="n.s."), size=0.1) + 
    geom_hline(yintercept=0) +
    ylim(-10, +10) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

jp_2 <- ggplot(mat_1 %>% dplyr::filter(celltype=="RPM"), aes(y=sign_log10_padj, x=dpi, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="RPM" & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="RPM" & sig!="n.s."), size=0.1) + 
    geom_hline(yintercept=0) +
    ylim(-10, +10) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

jp_3 <- ggplot(mat_1 %>% dplyr::filter(celltype=="Ery"), aes(y=sign_log10_padj, x=dpi, color=sig)) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="Ery" & sig=="n.s."), size=0.1) + 
    geom_jitter(data=mat_1 %>% dplyr::filter(celltype=="Ery" & sig!="n.s."), size=0.1) + 
    geom_hline(yintercept=0) +
    ylim(-10, +10) + 
    scale_color_manual(values=c("+/+"="#cd34b5", "cre/+"="#FFAC1E", "n.s."="#7f7f7f")) + 
    ylab("") + xlab("") + 
    facet_grid(~dpi, scales="free") + theme(aspect.ratio=2) + theme_global_set(4)

In [ ]:
counts_2 <- mat_1 %>% dplyr::filter(sig!="n.s.") %>% dplyr::group_by(dpi, celltype, sig) %>% dplyr::summarise(n=n()) %>% dplyr::arrange(celltype, dpi, sig)

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/jp_dea_count_genotype.pdf", width=3, height=4)

    gridExtra::grid.arrange(
            
        jp_1 %>% egg::set_panel_size(., width=unit(0.5, "cm"), height=unit(1.5, "cm")), 
        jp_2 %>% egg::set_panel_size(., width=unit(0.5, "cm"), height=unit(1.5, "cm")), 
        jp_3 %>% egg::set_panel_size(., width=unit(0.5, "cm"), height=unit(1.5, "cm")), 
        nrow=3
        
    )

dev.off()

# Gene set enrichment analysis (GSEA)

In [ ]:
adj_pval_thr <- 0.05
log2FC_thr <- 0.50

## Prepare results from genotype wald test

In [ ]:
res_dea_1 <- readRDS("result/dea/bulkRNAseq/res_dea_cmo.rds")
res_dea_2 <- readRDS("result/dea/bulkRNAseq/res_dea_rpm.rds")

In [ ]:
res_dea <- list("cMo"=res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]], "RPM"=res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]])

## Genotype comparison - HM

In [ ]:
gsea_res_1 <- lapply(seq_along(res_dea[[1]]), function(i) gsea(res_dea[[1]][[i]] %>% dplyr::rename(avg_log2FC=log2FoldChange, p_val_adj=ihw), category="MH", subcategory=NULL) %>% dplyr::mutate(col_split="cMo", col_label=paste0("group_", i)))
gsea_res_2 <- lapply(seq_along(res_dea[[2]]), function(i) gsea(res_dea[[2]][[i]] %>% dplyr::rename(avg_log2FC=log2FoldChange, p_val_adj=ihw), category="MH", subcategory=NULL) %>% dplyr::mutate(col_split="RPM", col_label=paste0("group_", i)))

In [ ]:
gsea_res_1 <- do.call("rbind", gsea_res_1)
gsea_res_2 <- do.call("rbind", gsea_res_2)

In [ ]:
gsea_res <- rbind(gsea_res_1, gsea_res_2)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05) %>% dplyr::pull(pathway) %>% unique()

In [ ]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [ ]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [ ]:
mat_1 <- mat_1[, c("col_label", "col_split", gs_name_select)]
mat_2 <- mat_2[, c("col_label", "col_split", gs_name_select)]

In [ ]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [ ]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_gsea_genotype_dpi_mh.pdf", width=8, height=50)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=NULL, pathway_suffix=NULL, color_neg=color$sample_group[1], color_pos=color$sample_group[2], fontsize_select=2)

dev.off()

## Genotype comparison - GOBP

In [ ]:
gsea_res_1 <- lapply(seq_along(res_dea[[1]]), function(i) gsea(res_dea[[1]][[i]] %>% dplyr::rename(avg_log2FC=log2FoldChange, p_val_adj=ihw), category="M5", subcategory="GO:BP") %>% dplyr::mutate(col_split="cMo", col_label=paste0("group_", i)))
gsea_res_2 <- lapply(seq_along(res_dea[[2]]), function(i) gsea(res_dea[[2]][[i]] %>% dplyr::rename(avg_log2FC=log2FoldChange, p_val_adj=ihw), category="M5", subcategory="GO:BP") %>% dplyr::mutate(col_split="RPM", col_label=paste0("group_", i)))

In [ ]:
gsea_res_1 <- do.call("rbind", gsea_res_1)
gsea_res_2 <- do.call("rbind", gsea_res_2)

In [ ]:
gsea_res <- rbind(gsea_res_1, gsea_res_2)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05) %>% dplyr::pull(pathway) %>% unique()

In [ ]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [ ]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [ ]:
mat_1 <- mat_1[, c("col_label", "col_split", gs_name_select)]
mat_2 <- mat_2[, c("col_label", "col_split", gs_name_select)]

In [ ]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [ ]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_gsea_genotype_dpi_gobp.pdf", width=8, height=50)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=NULL, pathway_suffix="GOBP", color_neg=color$sample_group[1], color_pos=color$sample_group[2], fontsize_select=2)

dev.off()

# DEA heatmap 

In [ ]:
adj_pval_thr <- 0.05
log2FC_thr <- 0.50

In [ ]:
hm_dea_fetch <- function(mat_1, dea_res, col_samples, col_split, fontsize_select, celltype, celltype_select, scale_mode="all") {

    mat_1 <- Seurat::CreateSeuratObject(counts=mat_1)
    mat_1 <- NormalizeData(mat_1, normalization.method="RC", scale.factor=1e6)
    mat_1 <- GetAssayData(mat_1, assay="RNA", layer="data") %>% as.data.frame() 

    if(scale_mode=="all") {

        mat_1 <- t(apply(mat_1, 1, scales::rescale))
        mat_1 <- mat_1[row_split, col_samples]

        colnames(mat_1) <- col_label

        row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))
        row_split <- row_split[row_split %in% rownames(mat_1)]
        
        mat_1 <- mat_1[row_split, celltype==celltype_select]
        col_split <- col_split[celltype==celltype_select]
        
    } else if (scale_mode=="sample") {

        mat_1 <- mat_1[row_split, col_samples]
        mat_1 <- mat_1[row_split, col_samples]

        colnames(mat_1) <- col_label

        row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))
        row_split <- row_split[row_split %in% rownames(mat_1)]
        
        mat_1 <- mat_1[row_split, celltype==celltype_select]
        col_split <- col_split[celltype==celltype_select]
        
        mat_1 <- t(apply(mat_1, 1, scales::rescale))
        
    }
    
    # DEA log2FC 
    mat_2 <- purrr::reduce(
        
        list(
            
            res_dea[[celltype_select]][[1]] %>% dplyr::mutate(Ctl=log2FoldChange) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, Ctl), 
            res_dea[[celltype_select]][[2]] %>% dplyr::mutate(D1=log2FoldChange) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D1), 
            res_dea[[celltype_select]][[3]] %>% dplyr::mutate(D3=log2FoldChange) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D3), 
            res_dea[[celltype_select]][[4]] %>% dplyr::mutate(D6=log2FoldChange) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D6)
        
        ), full_join, by="gene"
    
    ) %>% tibble::column_to_rownames("gene")

    mat_2[setdiff(row_split, rownames(mat_2)), ] <- 0
    mat_2 <- mat_2[row_split, ]
    mat_2[is.na(mat_2)] <- 0

    # DEA adj p-values
    mat_3 <- purrr::reduce(
        
        list(
            
            res_dea[[celltype_select]][[1]] %>% dplyr::mutate(Ctl=padj) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, Ctl), 
            res_dea[[celltype_select]][[2]] %>% dplyr::mutate(D1=padj) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D1), 
            res_dea[[celltype_select]][[3]] %>% dplyr::mutate(D3=padj) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D3), 
            res_dea[[celltype_select]][[4]] %>% dplyr::mutate(D6=padj) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D6)
        
        ), full_join, by="gene"
    
    ) %>% tibble::column_to_rownames("gene")

    mat_3[setdiff(row_split, rownames(mat_3)), ] <- 1
    mat_3 <- mat_3[row_split, ]
    mat_3[is.na(mat_3)] <- 1
    mat_3[(mat_3>adj_pval_thr & abs(mat_2)<log2FC_thr)] <- 1

    # Set log2FC to 0 if not significant 
    mat_2[abs(mat_2) < log2FC_thr] <- 0

    # Row split
    row_split=names(row_split)
    row_split=factor(row_split, levels=names(row_genes))
    
    hm_1 <- dea_exp_hm(mat_1, col_split=col_split, row_split=row_split, fontsize_select=2, breaks_limit=1, scale_width=0.5, legend_title="Exp.", cluster_rows=FALSE)
    hm_2 <- dea_res_hm(mat_2, mat_3, p_val_adj_thr=adj_pval_thr, row_split=row_split, scale_width=1, color_min="#cd34b5", color_max="#FFAC1E", cluster_rows=FALSE, fontsize_select=2)

    hm <- hm_2 + hm_1

    return(hm)
    
}

# Prepare meta data

In [ ]:
meta <- readRDS("data/bulkRNAseq/object/meta.rds") %>% dplyr::filter(celltype %in% c("cMo", "RPM"))

In [ ]:
meta <- meta %>% dplyr::arrange(celltype, dpi, genotype)

col_samples <- as.character(meta$sample_name)
col_label <- as.character(meta$genotype)
col_split <- as.character(meta$dpi)
celltype <- as.character(meta$celltype)

## Prepare results from genotype wald test

In [ ]:
res_dea_1 <- readRDS("result/dea/bulkRNAseq/res_dea_cmo.rds")
res_dea_2 <- readRDS("result/dea/bulkRNAseq/res_dea_rpm.rds")

In [ ]:
res_dea_1[["resc"]][["pc_wt_vs_ko"]] <- res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::mutate(response_class_wt=ifelse(response_gene_wt, response_class_wt, NA), response_class_ko=ifelse(response_gene_ko, response_class_ko, NA))
res_dea_2[["resc"]][["pc_wt_vs_ko"]] <- res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::mutate(response_class_wt=ifelse(response_gene_wt, response_class_wt, NA), response_class_ko=ifelse(response_gene_ko, response_class_ko, NA))

In [ ]:
res_dea_1[["resc"]][["pc_wt_vs_ko"]] <- res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter((response_gene_wt | response_gene_ko) & perturbation_gene) %>% dplyr::filter(!(is.na(response_class_wt) & is.na(response_class_ko)))
res_dea_2[["resc"]][["pc_wt_vs_ko"]] <- res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter((response_gene_wt | response_gene_ko) & perturbation_gene) %>% dplyr::filter(!(is.na(response_class_wt) & is.na(response_class_ko)))

In [ ]:
res_dea <- list("cMo"=res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]], "RPM"=res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]])

## Prepare expression mat

In [ ]:
meta <- readRDS("data/bulkRNAseq/object/meta.rds") %>% dplyr::filter(celltype %in% c("cMo", "RPM")) %>% droplevels()

In [ ]:
mat_1 <- readRDS("data/bulkRNAseq/object/cnt.rds") %>% dplyr::select(-mgi_symbol, -ensembl_gene_id)
mat_1 <- mat_1[, rownames(meta)]

In [ ]:
all(rownames(meta)==colnames(mat_1))

## cMo cre/+ UP vs +/+

In [ ]:
gene_select <- lapply(res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]], function(x) x %>% dplyr::filter(ihw<=adj_pval_thr & log2FoldChange>=log2FC_thr) %>% rownames())
gene_select <- do.call("c", gene_select) %>% unique()
gene_select <- gene_select[!gene_select %in% grep("Gm|Rps|Rpl", gene_select, value=TRUE)]

In [ ]:
row_genes <- list(

    "Repressed"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & response_class_ko=="Repressed") %>% rownames(), 
    "Activated"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & response_class_ko=="Activated") %>% rownames(), 
    "Activated (+/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & is.na(response_class_ko)) %>% rownames(), 
    "Activated (cre/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Activated") %>% rownames(), 
    "Repressed (+/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & is.na(response_class_ko)) %>% rownames(), 
    "Repressed (cre/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Repressed") %>% rownames(),
    "Opposed"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt!=response_class_ko) %>% rownames()
    
)

In [ ]:
row_genes <- lapply(row_genes, function(x) x[x %in% gene_select])

In [ ]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))

In [ ]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="cMo", scale_mode="sample")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_ko_up_vs_wt_cmo.pdf", width=10, height=70)

hm_1

dev.off()

## cMo cre/+ DOWN vs +/+

In [ ]:
gene_select <- lapply(res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]], function(x) x %>% dplyr::filter(ihw<=adj_pval_thr & log2FoldChange<=-log2FC_thr) %>% rownames())
gene_select <- do.call("c", gene_select) %>% unique()
gene_select <- gene_select[!gene_select %in% grep("Gm|Rps|Rpl", gene_select, value=TRUE)]

In [ ]:
row_genes <- list(

    "Repressed"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & response_class_ko=="Repressed") %>% rownames(), 
    "Activated"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & response_class_ko=="Activated") %>% rownames(), 
    "Activated (+/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & is.na(response_class_ko)) %>% rownames(), 
    "Activated (cre/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Activated") %>% rownames(), 
    "Repressed (+/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & is.na(response_class_ko)) %>% rownames(), 
    "Repressed (cre/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Repressed") %>% rownames(),
    "Opposed"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt!=response_class_ko) %>% rownames()
    
)

In [ ]:
row_genes <- lapply(row_genes, function(x) x[x %in% gene_select])

In [ ]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))

In [ ]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="cMo", scale_mode="sample")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_ko_down_vs_wt_cmo.pdf", width=10, height=80)

hm_1

dev.off()

## RPM cre/+ UP vs +/+

In [ ]:
gene_select <- lapply(res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]], function(x) x %>% dplyr::filter(ihw<=adj_pval_thr & log2FoldChange>=log2FC_thr) %>% rownames())
gene_select <- do.call("c", gene_select) %>% unique()
gene_select <- gene_select[!gene_select %in% grep("Gm|Rps|Rpl", gene_select, value=TRUE)]

In [ ]:
row_genes <- list(

    "Repressed"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & response_class_ko=="Repressed") %>% rownames(), 
    "Activated"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & response_class_ko=="Activated") %>% rownames(), 
    "Activated (+/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & is.na(response_class_ko)) %>% rownames(), 
    "Activated (cre/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Activated") %>% rownames(), 
    "Repressed (+/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & is.na(response_class_ko)) %>% rownames(), 
    "Repressed (cre/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Repressed") %>% rownames(),
    "Opposed"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt!=response_class_ko) %>% rownames()
    
)

In [ ]:
row_genes <- lapply(row_genes, function(x) x[x %in% gene_select])

In [ ]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))

In [ ]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="RPM", scale_mode="sample")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_ko_up_vs_wt_rpm.pdf", width=10, height=30)

hm_1

dev.off()

## RPM cre/+ DOWN vs +/+

In [ ]:
gene_select <- lapply(res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]], function(x) x %>% dplyr::filter(ihw<=adj_pval_thr & log2FoldChange<=-log2FC_thr) %>% rownames())
gene_select <- do.call("c", gene_select) %>% unique()
gene_select <- gene_select[!gene_select %in% grep("Gm|Rps|Rpl", gene_select, value=TRUE)]

In [ ]:
row_genes <- list(

    "Repressed"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & response_class_ko=="Repressed") %>% rownames(), 
    "Activated"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & response_class_ko=="Activated") %>% rownames(), 
    "Activated (+/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & is.na(response_class_ko)) %>% rownames(), 
    "Activated (cre/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Activated") %>% rownames(), 
    "Repressed (+/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & is.na(response_class_ko)) %>% rownames(), 
    "Repressed (cre/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Repressed") %>% rownames(),
    "Opposed"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt!=response_class_ko) %>% rownames()
    
)

In [ ]:
row_genes <- lapply(row_genes, function(x) x[x %in% gene_select])

In [ ]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))

In [ ]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="RPM", scale_mode="sample")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_ko_down_vs_wt_rpm.pdf", width=10, height=25)

hm_1

dev.off()

## Selected genes classical monocytes

In [ ]:
row_genes <- list(

    "G1"=c("Ddx60", "Ifih1", "Ifit1", "Ifit2", "Mx1", "Oasl1", "Oasl2", "Ccl2", "Cxcl10"), # Interferon stimulation 
    "G2"=c("Hmox1", "MMaf", "Mafb", "Vcam1", "Fcer1g", "Hck", "Prkcd", "C1qb", "C5ar1", "Msr1", "Axl"), # Macrophage development
    "G3"=c("Slc40a1", "Slc48a1", "Ftl1", "Fth1", "Hmox2", "Nfe2l2", "Acsl4", "Sod2", "Prdx1"), # Erythrophagocytosis 
    "G4"=c("Il1b", "Casp1", "Casp8", "Cflar"), # Inflammasome
    # "G5"=c("Tnfaip3", "Tnip1", "Il18bp", "Il10"), # Antinfommatory 
    "G6"=c("Atp5b", "Uqcrh", "Cox7a2l", "Sod1", "Gpx4", "Pon2"), # OXPHOS 
    "G7"=c("Psmb3", "Psmb5", "Psmb6", "Psmb8") # Proteasome 
    # "G8"=c("Ctse", "Ctsh", "Sting1") # Lysosome 
    
)

In [ ]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))
row_split[!row_split %in% rownames(mat_1)]

In [ ]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="cMo", scale_mode="sample")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_cmo_selected.pdf", width=10, height=10)

hm_1

dev.off()

## Selected genes classical RPMs

In [ ]:
row_genes <- list(

    "G1"=c("Stat2", "Irf7", "Ifit1", "Ifit2", "Ifitm3", "Ifi205", "Ifi206", "Ifi208", "Ifi211", "Usp18"), # IFN-response 
    "G2"=c("Eif2ak2", "Atf4", "Egr1", "Gdf15"), # Integrated stress response. Eif2ak2 is PKR, a kinase activator of 
    "G3"=c("Sod2", "Cflar", "Kitl", "Hif1a", "Ldha"), 
    "G4"=c("Top2a", "Ube2c", "Cdk1", "Mcm5")
)

In [ ]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))
row_split[!row_split %in% rownames(mat_1)]

In [ ]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="RPM", scale_mode="sample")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_rpm_selected.pdf", width=10, height=10)

hm_1

dev.off()

# PCA

In [ ]:
meta <- readRDS("data/bulkRNAseq/object/meta.rds") 
cnt <- readRDS("data/bulkRNAseq/object/cnt.rds") %>% dplyr::select(-mgi_symbol, -ensembl_gene_id)

## cMo 

In [ ]:
cnt_1 <- cnt[, meta$celltype=="cMo"]
meta_1 <- meta[meta$celltype=="cMo", ]
all(rownames(meta_1)==colnames(cnt_1))

In [ ]:
genes_1 <- c(
    
    res_dea_1[["resc"]][["rc_wt"]] %>% dplyr::filter(response_gene) %>% rownames(), 
    res_dea_1[["resc"]][["rc_ko"]] %>% dplyr::filter(response_gene) %>% rownames()

)

In [ ]:
cnt_1 <- cnt_1[rownames(cnt_1) %in% genes_1, ]

In [ ]:
dss <- DESeqDataSetFromMatrix(countData=cnt_1, colData=meta_1, design=~dpi)
vst <- DESeq2::vst(dss)
vst <- assay(vst)

In [ ]:
rv  <- rowVars(vst)
genes <- names(rv[order(rv, decreasing=TRUE)][1:100])

In [ ]:
pca <- prcomp(t(vst[genes, , drop=FALSE]), center=TRUE, scale.=FALSE)

In [ ]:
data <- cbind(pca$x[, c("PC1", "PC2")], meta_1)

In [ ]:
centroids <- data %>% group_by(dpi) %>% summarise(PC1=mean(PC1), PC2=mean(PC2), .groups="drop")
segments <- data %>% left_join(centroids, by = "dpi", suffix = c("", "_centroid"))

In [ ]:
pca_p <- ggplot(data, aes(x=PC1, y=PC2)) + 
    geom_point(data=centroids, size=1, shape=4, stroke=1.5) +
    geom_segment(data=segments,aes(xend=PC1_centroid, yend=PC2_centroid, group=dpi), alpha=0.4) +
    geom_point(aes(color=dpi, fill=dpi, shape=genotype), size=1, stroke=1) + 
    ggtitle("PCA (cMo)") + 
    xlab(paste0("PC1: ", round(100*(pca$sdev^2)/sum(pca$sdev^2))[1],"% variance")) +
    ylab(paste0("PC2: ", round(100*(pca$sdev^2)/sum(pca$sdev^2))[2],"% variance")) +
    scale_color_manual(values=color$dpi) + 
    theme(aspect.ratio=1, legend.position="right") + guides(fill=guide_legend(byrow=TRUE)) + guides(color=guide_legend(byrow=TRUE)) + guides(shape=guide_legend(byrow=TRUE)) + 
    theme_global_set(4)

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/pca_cmo.pdf", width=5, height=2)

pca_p

dev.off()

## RPM 

In [ ]:
cnt_1 <- cnt[, meta$celltype=="RPM"]
meta_1 <- meta[meta$celltype=="RPM", ]
all(rownames(meta_1)==colnames(cnt_1))

In [ ]:
genes_1 <- c(
    
    res_dea_2[["resc"]][["rc_wt"]] %>% dplyr::filter(response_gene) %>% rownames(), 
    res_dea_2[["resc"]][["rc_ko"]] %>% dplyr::filter(response_gene) %>% rownames()

)

In [ ]:
cnt_1 <- cnt_1[rownames(cnt_1) %in% genes_1, ]

In [ ]:
dss <- DESeqDataSetFromMatrix(countData=cnt_1, colData=meta_1, design=~dpi)
vst <- DESeq2::vst(dss)
vst <- assay(vst)

In [ ]:
rv  <- rowVars(vst)
genes <- names(rv[order(rv, decreasing=TRUE)][1:100])

In [ ]:
pca <- prcomp(t(vst[genes, , drop=FALSE]), center=TRUE, scale.=FALSE)

In [ ]:
data <- cbind(pca$x[, c("PC1", "PC2")], meta_1)

In [ ]:
centroids <- data %>% group_by(dpi) %>% summarise(PC1=mean(PC1), PC2=mean(PC2), .groups="drop")
segments <- data %>% left_join(centroids, by = "dpi", suffix = c("", "_centroid"))

In [ ]:
pca_p <- ggplot(data, aes(x=PC1, y=PC2)) + 
    geom_point(data=centroids, size=1, shape=4, stroke=1.5) +
    geom_segment(data=segments,aes(xend=PC1_centroid, yend=PC2_centroid, group=dpi), alpha=0.4) +
    geom_point(aes(color=dpi, fill=dpi, shape=genotype), size=1, stroke=1) + 
    ggtitle("PCA (cMo)") + 
    xlab(paste0("PC1: ", round(100*(pca$sdev^2)/sum(pca$sdev^2))[1],"% variance")) +
    ylab(paste0("PC2: ", round(100*(pca$sdev^2)/sum(pca$sdev^2))[2],"% variance")) +
    scale_color_manual(values=color$dpi) + 
    theme(aspect.ratio=1, legend.position="right") + guides(fill=guide_legend(byrow=TRUE)) + guides(color=guide_legend(byrow=TRUE)) + guides(shape=guide_legend(byrow=TRUE)) + 
    theme_global_set(4)

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/pca_rpm.pdf", width=5, height=2)

pca_p

dev.off()

# STRING 

In [ ]:
# Create STRING DB for mouse 
string_db <- STRINGdb::STRINGdb$new(version="12.0", species=10090, score_threshold=400, network_type="full", link_data="combined_only")

In [ ]:
res_dea_1 <- readRDS("result/dea/bulkRNAseq/res_dea_cmo.rds")
res_dea_2 <- readRDS("result/dea/bulkRNAseq/res_dea_rpm.rds")

## cMo D1 UP

In [ ]:
# Mapp DEA results to STRING DB
string_db_dea <- res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]][[2]] %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, log2FoldChange, ihw) %>% dplyr::rename(logFC=log2FoldChange, padj=ihw) %>% na.omit() %>% dplyr::filter(padj<=adj_pval_thr & logFC>=+log2FC_thr) %>% string_db$map(., "gene", removeUnmappedRows=TRUE)

In [ ]:
# Labe PPI vertices 
ppi_v <- string_db_dea %>% dplyr::distinct() %>% dplyr::rename(name=STRING_id) %>% 
    dplyr::group_by(name) %>% dplyr::mutate(n=n()) %>% dplyr::filter(n==1) %>% dplyr::select(name, gene, logFC, padj) %>% 
    dplyr::mutate(node_group=ifelse(padj<=0.05 & logFC<=-0.50, "neg", ifelse(padj<=0.05 & logFC>=0.50, "pos", "n.s.")))

In [ ]:
# Create PPI edges from DEA PPI
ppi_e <- string_db$get_interactions(string_db_dea$STRING_id) %>% dplyr::filter(from %in% ppi_v$name & to %in% ppi_v$name)

In [ ]:
# Compute igraph object
g <- igraph::graph_from_data_frame(
    
    d=ppi_e,
    vertices=ppi_v, 
    directed=FALSE

)

In [ ]:
# Connected components 
min_component_size <- 4  # keep components with >= 4 vertices

g_components <- igraph::components(g)
g <- igraph::delete_vertices(g, which(g_components$csize[g_components$membership] < min_component_size))

In [ ]:
# Compute Louvain cluster on combined score 
E(g)$weight <- E(g)$combined_score

louvain <- igraph::cluster_louvain(g, weights=E(g)$weight)
V(g)$louvain <- membership(louvain)

In [ ]:
# Compute GO enrichment for Louvain 
enrichment_louvain <- tibble::tibble(name=V(g)$name, louvain=V(g)$louvain) %>% group_by(louvain) %>%
    
    group_modify(~{
        
        enrichment <- string_db$get_enrichment(.x$name, category="Process", methodMT="fdr", iea=TRUE) %>% arrange(fdr)
        if (is.null(enrichment) | nrow(enrichment)==0) {
            
            return(tibble::tibble())
        
        } else {

            return(enrichment %>% arrange(fdr))
            
        }
        
    }
                ) %>% ungroup()

In [ ]:
# Fetch GO enrichment 
enrichment_label <- enrichment_louvain %>% group_by(louvain) %>% arrange(fdr) %>% slice(1) %>% ungroup() %>% transmute(louvain, label=description)
V(g)$enrichment_label <- enrichment_label$label[match(V(g)$louvain, enrichment_label$louvain)]

In [ ]:
# Remove vertices without enrichment label 
g <- igraph::delete_vertices(g, which(is.na(V(g)$enrichment_label)))

In [ ]:
# Plot graph 
g_1 <- tidygraph::as_tbl_graph(g) %>% ggraph::ggraph(., layout="lgl") +
    geom_edge_link(alpha=0.2, width=0.1, color="#000000") +
    geom_node_point(aes(fill=factor(enrichment_label), size=abs(logFC)), color="black", shape=21, stroke=0.1) +
    # geom_node_text(aes(label=ifelse(gene %in% c("Slc40a1", "Slc48a1", "Nr1h3", "Hmox1", "Stat2", "Irf7", "Eif2ak2", "Gdf15"), gene, NA)), repel=FALSE, size=2.1, color="black") + 
    # scale_fill_manual(values=c("#6d6875", "#4f6d7a", "#4a6fa5", "#161b33")) +
    scale_size_continuous(range=c(0.5, 3)) + 
    theme(aspect.ratio=1) + theme_global_set(4)

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/string_cmo_d1_ko_up.pdf", width=10, height=10)

gridExtra::grid.arrange(egg::set_panel_size(g_1, width=unit(5, "cm"), height=unit(5, "cm")))

dev.off()

## cMo D1 DOWN

In [ ]:
# Mapp DEA results to STRING DB
string_db_dea <- res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]][[2]] %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, log2FoldChange, ihw) %>% dplyr::rename(logFC=log2FoldChange, padj=ihw) %>% na.omit() %>% dplyr::filter(padj<=adj_pval_thr & logFC<=-log2FC_thr) %>% string_db$map(., "gene", removeUnmappedRows=TRUE)

In [ ]:
# Labe PPI vertices 
ppi_v <- string_db_dea %>% dplyr::distinct() %>% dplyr::rename(name=STRING_id) %>% 
    dplyr::group_by(name) %>% dplyr::mutate(n=n()) %>% dplyr::filter(n==1) %>% dplyr::select(name, gene, logFC, padj) %>% 
    dplyr::mutate(node_group=ifelse(padj<=0.05 & logFC<=-0.50, "neg", ifelse(padj<=0.05 & logFC>=0.50, "pos", "n.s.")))

In [ ]:
# Create PPI edges from DEA PPI
ppi_e <- string_db$get_interactions(string_db_dea$STRING_id) %>% dplyr::filter(from %in% ppi_v$name & to %in% ppi_v$name)

In [ ]:
# Compute igraph object
g <- igraph::graph_from_data_frame(
    
    d=ppi_e,
    vertices=ppi_v, 
    directed=FALSE

)

In [ ]:
# Connected components 
min_component_size <- 4  # keep components with >= 4 vertices

g_components <- igraph::components(g)
g <- igraph::delete_vertices(g, which(g_components$csize[g_components$membership] < min_component_size))

In [ ]:
# Compute Louvain cluster on combined score 
E(g)$weight <- E(g)$combined_score

louvain <- igraph::cluster_louvain(g, weights=E(g)$weight)
V(g)$louvain <- membership(louvain)

In [ ]:
# Compute GO enrichment for Louvain 
enrichment_louvain <- tibble::tibble(name=V(g)$name, louvain=V(g)$louvain) %>% group_by(louvain) %>%
    
    group_modify(~{
        
        enrichment <- string_db$get_enrichment(.x$name, category="Process", methodMT="fdr", iea=TRUE) %>% arrange(fdr)
        if (is.null(enrichment) | nrow(enrichment)==0) {
            
            return(tibble::tibble())
        
        } else {

            return(enrichment %>% arrange(fdr))
            
        }
        
    }
                ) %>% ungroup()

In [ ]:
# Fetch GO enrichment 
enrichment_label <- enrichment_louvain %>% group_by(louvain) %>% arrange(fdr) %>% slice(1) %>% ungroup() %>% transmute(louvain, label=description)
V(g)$enrichment_label <- enrichment_label$label[match(V(g)$louvain, enrichment_label$louvain)]

In [ ]:
# Remove vertices without enrichment label 
g <- igraph::delete_vertices(g, which(is.na(V(g)$enrichment_label)))

In [ ]:
# Plot graph 
g_1 <- tidygraph::as_tbl_graph(g) %>% ggraph::ggraph(., layout="lgl") +
    geom_edge_link(alpha=0.2, width=0.1, color="#000000") +
    geom_node_point(aes(fill=factor(enrichment_label), size=abs(logFC)), color="black", shape=21, stroke=0.1) +
    # geom_node_text(aes(label=ifelse(gene %in% c("Slc40a1", "Slc48a1", "Nr1h3", "Hmox1", "Stat2", "Irf7", "Eif2ak2", "Gdf15"), gene, NA)), repel=FALSE, size=2.1, color="black") + 
    # scale_fill_manual(values=c("#6d6875", "#4f6d7a", "#4a6fa5", "#161b33")) +
    scale_size_continuous(range=c(0.5, 3)) + 
    theme(aspect.ratio=1) + theme_global_set(4)

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/string_cmo_d1_ko_down.pdf", width=10, height=10)

gridExtra::grid.arrange(egg::set_panel_size(g_1, width=unit(5, "cm"), height=unit(5, "cm")))

dev.off()

## RPM D1 UP

In [ ]:
# Mapp DEA results to STRING DB
string_db_dea <- res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]][[2]] %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, log2FoldChange, ihw) %>% dplyr::rename(logFC=log2FoldChange, padj=ihw) %>% na.omit() %>% dplyr::filter(padj<=adj_pval_thr & logFC>=+log2FC_thr) %>% string_db$map(., "gene", removeUnmappedRows=TRUE)

In [ ]:
# Labe PPI vertices 
ppi_v <- string_db_dea %>% dplyr::distinct() %>% dplyr::rename(name=STRING_id) %>% 
    dplyr::group_by(name) %>% dplyr::mutate(n=n()) %>% dplyr::filter(n==1) %>% dplyr::select(name, gene, logFC, padj) %>% 
    dplyr::mutate(node_group=ifelse(padj<=0.05 & logFC<=-0.50, "neg", ifelse(padj<=0.05 & logFC>=0.50, "pos", "n.s.")))

In [ ]:
# Create PPI edges from DEA PPI
ppi_e <- string_db$get_interactions(string_db_dea$STRING_id) %>% dplyr::filter(from %in% ppi_v$name & to %in% ppi_v$name)

In [ ]:
# Compute igraph object
g <- igraph::graph_from_data_frame(
    
    d=ppi_e,
    vertices=ppi_v, 
    directed=FALSE

)

In [ ]:
# Connected components 
min_component_size <- 4  # keep components with >= 4 vertices

g_components <- igraph::components(g)
g <- igraph::delete_vertices(g, which(g_components$csize[g_components$membership] < min_component_size))

In [ ]:
# Compute Louvain cluster on combined score 
E(g)$weight <- E(g)$combined_score

louvain <- igraph::cluster_louvain(g, weights=E(g)$weight)
V(g)$louvain <- membership(louvain)

In [ ]:
# Compute GO enrichment for Louvain 
enrichment_louvain <- tibble::tibble(name=V(g)$name, louvain=V(g)$louvain) %>% group_by(louvain) %>%
    
    group_modify(~{
        
        enrichment <- string_db$get_enrichment(.x$name, category="Process", methodMT="fdr", iea=TRUE) %>% arrange(fdr)
        if (is.null(enrichment) | nrow(enrichment)==0) {
            
            return(tibble::tibble())
        
        } else {

            return(enrichment %>% arrange(fdr))
            
        }
        
    }
                ) %>% ungroup()

In [ ]:
# Fetch GO enrichment 
enrichment_label <- enrichment_louvain %>% group_by(louvain) %>% arrange(fdr) %>% slice(1) %>% ungroup() %>% transmute(louvain, label=description)
V(g)$enrichment_label <- enrichment_label$label[match(V(g)$louvain, enrichment_label$louvain)]

In [ ]:
# Remove vertices without enrichment label 
g <- igraph::delete_vertices(g, which(is.na(V(g)$enrichment_label)))

In [ ]:
# Plot graph 
g_1 <- tidygraph::as_tbl_graph(g) %>% ggraph::ggraph(., layout="lgl") +
    geom_edge_link(alpha=0.2, width=0.1, color="#000000") +
    geom_node_point(aes(fill=factor(enrichment_label), size=abs(logFC)), color="black", shape=21, stroke=0.1) +
    # geom_node_text(aes(label=ifelse(gene %in% c("Slc40a1", "Slc48a1", "Nr1h3", "Hmox1", "Stat2", "Irf7", "Eif2ak2", "Gdf15"), gene, NA)), repel=FALSE, size=2.1, color="black") + 
    # scale_fill_manual(values=c("#6d6875", "#4f6d7a", "#4a6fa5", "#161b33")) +
    scale_size_continuous(range=c(0.5, 3)) + 
    theme(aspect.ratio=1) + theme_global_set(4)

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/string_rpm_d1_ko_up.pdf", width=10, height=10)

gridExtra::grid.arrange(egg::set_panel_size(g_1, width=unit(5, "cm"), height=unit(5, "cm")))

dev.off()

## RPM D1 UP

In [ ]:
# Mapp DEA results to STRING DB
string_db_dea <- res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]][[2]] %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, log2FoldChange, ihw) %>% dplyr::rename(logFC=log2FoldChange, padj=ihw) %>% na.omit() %>% dplyr::filter(padj<=adj_pval_thr & logFC<=-log2FC_thr) %>% string_db$map(., "gene", removeUnmappedRows=TRUE)

In [ ]:
# Labe PPI vertices 
ppi_v <- string_db_dea %>% dplyr::distinct() %>% dplyr::rename(name=STRING_id) %>% 
    dplyr::group_by(name) %>% dplyr::mutate(n=n()) %>% dplyr::filter(n==1) %>% dplyr::select(name, gene, logFC, padj) %>% 
    dplyr::mutate(node_group=ifelse(padj<=0.05 & logFC<=-0.50, "neg", ifelse(padj<=0.05 & logFC>=0.50, "pos", "n.s.")))

In [ ]:
# Create PPI edges from DEA PPI
ppi_e <- string_db$get_interactions(string_db_dea$STRING_id) %>% dplyr::filter(from %in% ppi_v$name & to %in% ppi_v$name)

In [ ]:
# Compute igraph object
g <- igraph::graph_from_data_frame(
    
    d=ppi_e,
    vertices=ppi_v, 
    directed=FALSE

)

In [ ]:
# Connected components 
min_component_size <- 4  # keep components with >= 4 vertices

g_components <- igraph::components(g)
g <- igraph::delete_vertices(g, which(g_components$csize[g_components$membership] < min_component_size))

In [ ]:
# Compute Louvain cluster on combined score 
E(g)$weight <- E(g)$combined_score

louvain <- igraph::cluster_louvain(g, weights=E(g)$weight)
V(g)$louvain <- membership(louvain)

In [ ]:
# Compute GO enrichment for Louvain 
enrichment_louvain <- tibble::tibble(name=V(g)$name, louvain=V(g)$louvain) %>% group_by(louvain) %>%
    
    group_modify(~{
        
        enrichment <- string_db$get_enrichment(.x$name, category="Process", methodMT="fdr", iea=TRUE) %>% arrange(fdr)
        if (is.null(enrichment) | nrow(enrichment)==0) {
            
            return(tibble::tibble())
        
        } else {

            return(enrichment %>% arrange(fdr))
            
        }
        
    }
                ) %>% ungroup()

In [ ]:
# Fetch GO enrichment 
enrichment_label <- enrichment_louvain %>% group_by(louvain) %>% arrange(fdr) %>% slice(1) %>% ungroup() %>% transmute(louvain, label=description)
V(g)$enrichment_label <- enrichment_label$label[match(V(g)$louvain, enrichment_label$louvain)]

In [ ]:
# Remove vertices without enrichment label 
g <- igraph::delete_vertices(g, which(is.na(V(g)$enrichment_label)))

In [ ]:
# Plot graph 
g_1 <- tidygraph::as_tbl_graph(g) %>% ggraph::ggraph(., layout="lgl") +
    geom_edge_link(alpha=0.2, width=0.1, color="#000000") +
    geom_node_point(aes(fill=factor(enrichment_label), size=abs(logFC)), color="black", shape=21, stroke=0.1) +
    # geom_node_text(aes(label=ifelse(gene %in% c("Slc40a1", "Slc48a1", "Nr1h3", "Hmox1", "Stat2", "Irf7", "Eif2ak2", "Gdf15"), gene, NA)), repel=FALSE, size=2.1, color="black") + 
    # scale_fill_manual(values=c("#6d6875", "#4f6d7a", "#4a6fa5", "#161b33")) +
    scale_size_continuous(range=c(0.5, 3)) + 
    theme(aspect.ratio=1) + theme_global_set(4)

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/string_rpm_d1_ko_down.pdf", width=10, height=10)

gridExtra::grid.arrange(egg::set_panel_size(g_1, width=unit(5, "cm"), height=unit(5, "cm")))

dev.off()

# Session Info

In [ ]:
sessionInfo()